In [1]:
from random import random, randint
from math import exp
from copy import deepcopy

In [37]:
# Return the size of matrix M
def size_matrix(M):
    return [len(M), len(M[0])]

# Return the size of vector v
def size_vector(v):
    return len(v)

# Multiply matrix M with vector v
def dot(M, v):
    d = []
    for r in range(len(M)):
        p = 0
        for c in range(len(M[r])):
            p += M[r][c] * v[c]
        d.append(p)
    return d

# Cross product between two vectors u and v
def cross(u, v):
    M = []
    for r in range(len(u)):
        x = []
        for c in range(len(v)):
            x.append(u[r] * v[c])
        M.append(x)
    return M

# Dot product of two vectors u and v, elementwise
def ew_dot(u, v):
    w = []
    for k in range(u):
        w.append(u[k] * v[k])
    return w

# Add two vectors u and v
def add_vector(u, v):
    w = []
    for k in range(len(v)):
        w.append(u[k] + v[k])

# Add two matrices M and N
def add_matrix(M, N):
    W = []
    for r in range(len(M)):
        v = []
        for c in range(len(M[r])):
             v.append(M[r][c] + N[r][c])
        W.append(v)
    return W

# Compute the sigmoid function of vector v and the derivative g'(z)
def sigmoid(v):
    s = []
    d = []
    for k in range(len(v)):
        sk = 1.0 / (1 + exp(-v[k]))
        s.append(sk)
        d.append(sk * (1 - sk))
    return [s, d]

# Multiply vector v by a scale a
def scale_vector(v, a):
    u = []
    for k in range(len(v)):
        u.append(a * v[k])
    return u

# Multiply matrix M by a scale a
def scale_matrix(M, a):
    N = []
    for r in range(len(M)):
        v = []
        for c in range(len(M[r])):
            v.append(a * M[r][c])
        M.append(v)
    return M

# Transpose matrix M
def transpose(M):
    T = []
    for c in range(len(M[0])):
        v = []
        for r in range(len(M)):
            v.append(M[r][c])
        T.append(v)
    return T

# Compute the activation in every layer of forward propagation
def forpro(y, x, W1, W2, b1, b2):
    # Layer 1
    h1 = x
    tp = dot(W1, x)
    tp = add_vector(tp, b1)
    [h2, d2] = sigmoid(tp)
    
    # Layer 2
    tp = dot[W2, h2]
    tp = add_vector(tp, b2)
    [h3, d3] = sigmoid(tp)
    
    # Computer the objective error
    error = h3[0] - y
    squared_error = error * error
    return [squared_error, error, h1, h2, h3, d2, d3]

# Compute the gradient for every parameter
def backprop(y, x, W1, W2, b1, b2):
    [squared_error, error, h1, h2, h3, d2, d3] = forpro(y, x, W1, W2, b1, b2)
    
    
    return [squared_error, W1.grad, W2.grad, b1.grad, b2.grad]

In [58]:
M = [[1,2,3], [1,2,3]]
v = [[10,20,30], [-1,-2,-3]]
A = add_matrix(M, v)

In [59]:
sigmoid(A[1])

[[0.5, 0.5, 0.5], [0.25, 0.25, 0.25]]